In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
# chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [3]:
url='http://www.npes.org/news-and-events/calendar-of-events.aspx'

driver.get(url)
time.sleep(5)

soup=bs(driver.page_source,'lxml')
items=soup.find('div',{'class':'publishTextHtmlDisplay Normal'}).find_all('h2')

cards=[]
for a in items:
    title=' '.join(a.text.split())
    det=a.find_next("p")
    dem=' '.join(det.get_text(separator='{').split()).split('{')
    dat=dem[0].strip()
    
    
    loc=dem[1].strip()
    try:
        link=det.find('a')['href']
        if not link.startswith('http'):
            link='http://www.npes.org'+link
    except:
        link=''
    if '@' in link:
        link=''
    
    cards.append([title,dat,link,loc])


In [29]:
npes=[]
for w in cards:
    title=w[0]
    dat=w[1]
    link=w[2]
    
    loc=w[3]
    dat=dat.replace('–','-').replace('|','').replace('to','-').replace('TO','-').replace('c-ber','ctober').replace('C-BER','CTOBER').strip()
    for f in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','Mon','Tue','Wed','Thu','Fri','Sat','Sun']:
        if f in dat:
            dat=dat.replace(f,'').strip()
    
    if dat=='':
        start_date=end_date=''
    else:
        mad=dat.strip()
    if '-' in mad:
        st=mad.split('-')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('-')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif 'and'in mad:
        st=mad.split('and')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
        en=mad.split('and')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()

    elif '&' in mad:
        st=mad.split('&')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
        en=mad.split('&')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()

    else:
        st=mad.strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
        en=mad.strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
   
    if any(c.isalpha() for c in st)==False:
    #      print('leg')
        pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en
        
        
    if any(c.isalpha() for c in en)==False:
        if not re.search('\d{4}',st):
            start=re.search('\d{1,2}',st).group()+' '+re.search('[A-Za-z]+',st).group()+' '+str(datetime.today().year)
        else:
            start=re.search('\d{1,2}',st).group()+' '+re.search('[A-Za-z]+',st).group()+' '+re.search('\d{4}',st).group()
        if re.search('[A-Za-z]',st):
            if not re.search('\d{4}',st):
                st=st+' '+str(datetime.today().year)
        end=en+' '+re.search('[A-Za-z]+',st).group()+' '+str(datetime.today().year)
        
        
        
        
    #                        02        April      2022
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
     #   print('awujale')
        start=st
        end=en

    #*                        April       02      2022
    elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
    #       print('ala')
        pa=re.search('(\d{4})',st).group()
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()
        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba     

    #                      02     2022
    elif re.search('\d{1,2}\s+\d{4}',en):
    #      print('kum')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',st).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    #                      02         April
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
    #        print('is')
        pa=re.search('(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en

    #*                      April           02
    elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
    #      print('bad')
        if not re.search('(\d{4})',en):
            en=en+' '+str(datetime.today().year)
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    else:
    #     print('shik')
        start=end=''
    if start=='':
        start_date=end_date=''
    else:
        pick=[start, end]
        try:
            spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
        except:
            spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]
    
    
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            try:
                return(gu(location))
            except:
                sha=location.split(',')
                try:
                    gu(sha[-3])
                except:
                    try:
                        gu(sha[-2])
                    except:
                        try:
                            gu(sha[-1])
                        except Exception as e:
                            print(location, "; url didn't go through")
                            print(e)
                            return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                    except:
                        google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                        time.sleep(randint(0,3))
                        driver.get(google_url_for_location)
                        time.sleep(4)
                        google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")
        
    time_=''
    org_name="Association for Print Technologies"
    org_web='http://www.npes.org/'
    org_pro="The Association for Print Technologies (APTech℠) (formerly known as NPES), is a purpose-driven organization dedicated to supporting the entire commercial printing value chain. We believe in helping our members maximize alignment with their value chain partners to be more effective and competitive."
    logo='' 
    agendalist=''
    sponsor=''
    Speakerlist=''
    type_=''
    category=''
    Speakerlist=''
    ticket_list=''
    event_info=''
    event_web=link
    mail_=json.dumps(['chawkins@aptech.org'])

    
    if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc or 'ebinar' in loc:
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        
        on_off='0'
        if ',' in loc:
            city=loc.split(',')[0].strip()
            country=loc.split(',')[-1].strip()
        else:
            city=loc
            country=countr(city)
            
        venue=loc
        googlePlaceUrl=get_google_map_url(venue)
    
    
    npes.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])

ERROR! Session/line number was not unique in database. History logging moved to new session 1124
Berlin, Germany ; url didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":".Lx2b0d"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00AF6463+2188387]
	Ordinal0 [0x00A8E461+1762401]
	Ordinal0 [0x009A3D78+802168]
	Ordinal0 [0x009D1880+989312]
	Ordinal0 [0x009D1B1B+989979]
	Ordinal0 [0x009FE912+1173778]
	Ordinal0 [0x009EC824+1099812]
	Ordinal0 [0x009FCC22+1166370]
	Ordinal0 [0x009EC5F6+1099254]
	Ordinal0 [0x009C6BE0+945120]
	Ordinal0 [0x009C7AD6+948950]
	GetHandleVerifier [0x00D971F2+2712546]
	GetHandleVerifier [0x00D8886D+2652765]
	GetHandleVerifier [0x00B8002A+520730]
	GetHandleVerifier [0x00B7EE06+516086]
	Ordinal0 [0x00A9468B+1787531]
	Ordinal0 [0x00A98E88+1805960]
	Ordinal0 [0x00A98F75+1806197]
	Ordinal0 [0x00AA1DF1+1842673]
	BaseThreadInitThunk [0x75E7FA29+25]
	RtlGetAppContainerNamedObjectPath [0x773F7A9E

In [30]:
len(npes)

15

In [31]:
npes

[['',
  'TC130 Meeting',
  '2022-04-16',
  '2022-04-20',
  '',
  '',
  '',
  'The Association for Print Technologies (APTech℠) (formerly known as NPES), is a purpose-driven organization dedicated to supporting the entire commercial printing value chain. We believe in helping our members maximize alignment with their value chain partners to be more effective and competitive.',
  'Association for Print Technologies',
  'http://www.npes.org/',
  '',
  '',
  '',
  '',
  '',
  'Berlin',
  'Germany',
  'Berlin, Germany',
  '',
  '',
  '["chawkins@aptech.org"]',
  '',
  '0'],
 ['http://www.npes.org/programs/standardsworkroom/upcomingevents/isotc130wg5frankfurt.aspx',
  'TC130 Meeting',
  '2022-04-24',
  '2022-04-26',
  '',
  '',
  '',
  'The Association for Print Technologies (APTech℠) (formerly known as NPES), is a purpose-driven organization dedicated to supporting the entire commercial printing value chain. We believe in helping our members maximize alignment with their value chain partner

In [32]:
npes_dict=npes


npes_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=npes_dict)
npes_df.to_csv("npes.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [ ]:
# driver.switch_to.window(driver.window_handles[0])

In [ ]:
# unox=[]
# for k in range(1,31):
#     driver.switch_to.window(driver.window_handles[k])

#     soupn=bs(driver.page_source,'lxml')
#     title=' '.join(soupn.find('header',{'class':'main-title'}).text.split())
#     try:
#         dat=' '.join(soupn.find('div',{'class':re.compile('field field-name-field-event-date.+')}).find('span',{'class':'field-item-single'}).text.split())
#     except:
#         dat=''
    
#     dat=dat.replace('–','-').replace('|','').replace('to','-').replace('TO','-').strip()
#     if '(' in dat:
#         dat=dat.split('(')[0].strip()
#     if dat=='':
#         start_date=end_date=''
#     else:
#         mad=dat
#     if '-' in mad:
#         st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#         en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#     elif 'and'in mad:
#         st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#         en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#     elif '&' in mad:
#         st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#         en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#     else:
#         st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#         en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
   
#     if any(c.isalpha() for c in st)==False:
#     #      print('leg')
#         pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
#         sapa=pa.group()
#         start=st+' '+sapa
#         end=en
#     #                        02        April      2022
#     elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
#      #   print('awujale')
#         start=st
#         end=en

#     #*                        April       02      2022
#     elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
#     #       print('ala')
#         pa=re.search('(\d{4})',st).group()
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()
#         start=ta+' '+sa+' '+pa
#         #
#         ba=re.search('(\d{4})',en).group()
#         ca=re.search('([A-Sa-z]{3,9})',en).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba     

#     #                      02     2022
#     elif re.search('\d{1,2}\s+\d{4}',en):
#     #      print('kum')
#         pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()

#         start=ta+' '+sa+' '+pa
#         #
#         ba=re.search('(\d{4})',en).group()
#         ca=re.search('([A-Sa-z]{3,9})',st).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba

#     #                      02         April
#     elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
#     #        print('is')
#         pa=re.search('(\d{4})',en)#, maxsplit=0)
#         sapa=pa.group()
#         start=st+' '+sapa
#         end=en

#     #*                      April           02
#     elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
#     #      print('bad')
#         pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()

#         start=ta+' '+sa+' '+pa
#         #
#         ba=re.search('(\d{4})',en).group()
#         ca=re.search('([A-Sa-z]{3,9})',en).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba

#     else:
#     #     print('shik')
#         start=end=''
#     if start=='':
#         start_date=end_date=''
#     else:
#         pick=[start, end]
#         try:
#             spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
#         except:
#             spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
#         date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
#         start_date=date_[0]
#         end_date=date_[1]

   
#     try:
#         tim=' '.join(soupn.find('div',{'class':re.compile('field field-name-field-event-time.+')}).find('span',{'class':'field-item-single'}).text.split()).upper().replace('TO','-').split('-')#.strip()
#     except:
#         tim=''
#     try:
#         tic=' '.join(soupn.find('div',{'class':re.compile('field field-name-field-event-cost.+')}).find('span',{'class':'field-item-single'}).text.split())
#     except:
#         tic=''
#     if tic!='':
#         if 'Free' in tic or tic=='Free':
#             ty='free'
#             am=''
#             cu=''

#         elif re.search('\$?\£?\€?\W?[0-9]+',tic).group():
#             ty='paid'
#             am=re.search('[0-9]+',tic).group()
#             cu=re.search('\$?\£?\€?',tic).group()
#         ticket=[{'type': ty,
#                  'price':am,
#                  'currency': cu}]
#         ticket_list=json.dumps(ticket,ensure_ascii=False)
#     else:
#         ticket_list=''
    
    
#     try:
#         event_info=' '.join(soupn.find('div',{'class':re.compile('field field-name-field-body.+')}).find('span',{'class':'field-item-single'}).text.split())
#     except:
#         event_info=''
#     try:
#         event_web=soupn.find('div',{'class':re.compile('field field-name-field-further-information.+')}).find('a')['href']
#     except:
#         event_web=''

        
#     if tim=='':
#         time_=''
        
#     else:
#         #print('azure')
#         if len(tim)==2:
#             stt=datetime.strptime(tim[0].strip(), '%H:%M').strftime('%I:%M%p')
#             ent=datetime.strptime(tim[1].strip(), '%H:%M').strftime('%I:%M%p')
#             z=''
#         elif len(tim)==1:
#             stt=datetime.strptime(tim[0].strip(), '%H:%M').strftime('%I:%M%p')
#             ent=''
#             z=''
#         time_st={   
#             'type':'general',
#             'Start_time':stt,
#             'end_time':ent,
#             'timezone':z,
#             'days':'all'
#         }

#         time_=json.dumps([time_st])
# #     #**************
#     def gu(luc):
#         google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
#         time.sleep(randint(0,3))
#         driver.get(google_url_for_location)
#         time.sleep(4)
#         try:
#             google_map_url=driver.find_element_by_id('lu_map').click()
#         except:
#             try:
#                 google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
#             except:
#                 google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
#         time.sleep(1)
#         google_map_url=driver.current_url
# #                 print(google_map_url)
#         return(google_map_url)
#         ######################################
#     def get_google_map_url(location):
#         try:
#             return(gu(location))
#         except:
#             try:
#                 return(gu(location+', University of Michigan'))
#             except:
#                 sha=location.split(',')
#                 try:
#                     gu(sha[-3])
#                 except:
#                     try:
#                         gu(sha[-2])
#                     except:
#                         try:
#                             gu(sha[-1])
#                         except Exception as e:
#                             print(location, "; url didn't go through")
#                             print(e)
#                             return("")


#     def countr(locale):
#         try:
#             google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
#             time.sleep(randint(0,3))
#             driver.get(google_url_for_location)
#             time.sleep(4)
#             try:
#                 google_map_url=driver.find_element_by_id('lu_map').click()
#             except:
#                 try:
#                     google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
#                 except:
#                     try:
#                         google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
#                     except:
#                         locale=locale+', University of Michigan'
#                         google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
#                         time.sleep(randint(0,3))
#                         driver.get(google_url_for_location)
#                         time.sleep(4)
#                         google_map_url=driver.find_element_by_id('lu_map').click()
#             time.sleep(1)
#             try:
#                 country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
#             except:
#                 country=driver.find_element_by_class_name('bwoZTb').text
# #                 print(country)
#             return(country)
#         except Exception as e:
#             print(locale, ";country didn't go through")
#             print(e)
#             return("")

#     try:
#         loc=' '.join(soupn.find('div',{'class':re.compile('field field-name-field-event-venue.+')}).find('span',{'class':'field-item-single'}).text.split())
#     except:
#         loc=''

#     org_name='University of Oxford'
#     org_web='https://www.ox.ac.uk/'
#     org_pro='Oxford has a distinctive collegiate structure. Students and academics benefit from belonging both to the University, a large, internationally-renowned institution, and to a college or hall, a small, interdisciplinary academic community.'
#     logo=''    
#     sponsor=''
#     agendalist=''
#     type_=''
#     category=''
#     Speakerlist='' 

#     try:
#         mail_=json.dumps[' '.join(soupn.find('div',{'class':re.compile('field field-name-field-contact-email.+')}).text.split())]
#     except:
#         mail_=''
#     if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc:
#         on_off='1'
#         city=''
#         venue=''
#         country=''
#         googlePlaceUrl=''
#     else:
#         on_off='0'
#         if ',' in loc:
#             city=loc.split(',')[-2].strip()
#             country=loc.split(',')[-1].strip()
#         else:
#             city=loc.split()[-3]
#             country=countr(city)
#         venue=loc
#         googlePlaceUrl=get_google_map_url(venue)


#     unox.append([link,title,start_date,end_date,time_,event_info,ticket_list,
#                  org_pro,org_name,org_web,logo,sponsor,agendalist,
#                  type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
#                  Speakerlist,on_off])

# unox    
# #print('Done')

In [ ]:
# s=1
# for k in cards:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[0])
#     s+=1
# print(s+1)

In [ ]:
# for k in range(1,31):
#     time.sleep(1)
#     driver.switch_to.window(driver.window_handles[k])
